In [1]:
# !pip install tensorflow
# !pip install keras

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import random
from tensorflow import keras
import numpy as np
# from tensorflow.keras import layers, Dense, Input, InputLayer, Flatten
# from tensorflow.keras.models import Sequential, Model

from keras import layers
from keras.layers import Dense,Input, InputLayer, Flatten


from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using TensorFlow backend.


In [3]:
# img_folder=r'/content/drive/MyDrive/Major Project/Full Dataset'
img_folder = r'./Datasets/training-data/'

In [4]:
# image = cv2.imread(r'./Datasets/Test/bright_dune/ESP_016576_2570_RED-0003.jpg')
# image = np.array(image)
# image = image.astype('float32')
# image /= 255
# print(image.shape)

In [ ]:
def create_dataset(img_folder):

    img_data_array = list()
    class_name=[]
    
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            image = cv2.imread(image_path)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
            print(len(class_name))
    return img_data_array, class_name

# extract the image array and class name
img_data, class_name = create_dataset(img_folder)

In [6]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_dict

{'bright_dune': 0,
 'crater': 1,
 'dark_dune': 2,
 'impact_ejecta': 3,
 'slope_streak': 4}

In [7]:
target_val =  [target_dict[class_name[i]] for i in range(len(class_name))]

In [8]:
print(len(target_val), len(img_data))

11919 11919


In [9]:
model=tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(227,227, 3)),
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(5)
        ])

In [10]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(x=np.array(img_data, np.float32), y=np.array(list(map(int,target_val)), np.float32), epochs=15)

Train on 11919 samples
Epoch 1/15
11919/11919 [==============================] - 20s 2ms/sample - loss: 5.2658 - accuracy: 0.1381
Epoch 2/15
11919/11919 [==============================] - 11s 895us/sample - loss: 4.9987 - accuracy: 0.1030
Epoch 3/15
11919/11919 [==============================] - 11s 947us/sample - loss: 4.9987 - accuracy: 0.1030
Epoch 4/15
11919/11919 [==============================] - 12s 978us/sample - loss: 4.9987 - accuracy: 0.1030
Epoch 5/15
11919/11919 [==============================] - 11s 938us/sample - loss: 4.9987 - accuracy: 0.1030
Epoch 6/15
11919/11919 [==============================] - 11s 942us/sample - loss: 4.9987 - accuracy: 0.1030
Epoch 7/15
11919/11919 [==============================] - 12s 1ms/sample - loss: 4.9987 - accuracy: 0.1030
Epoch 8/15
11919/11919 [==============================] - 12s 1ms/sample - loss: 4.9987 - accuracy: 0.1030
Epoch 9/15
11919/11919 [==============================] - 11s 885us/sample - loss: 4.9987 - accuracy: 0.1030
Ep

In [12]:
prediction_class = {0: 'bright_dune', 1: 'crater', 2: 'dark_dune', 3: 'impact_ejecta', 4: 'slope_streak'}

In [13]:
import pickle
model.save('models/cnn-1.h5')

In [14]:
def findClass(image_path):
    image_1 = cv2.imread(image_path)
    image_1 = np.array(image_1)
    image_1 = image_1.astype('float32')
    image_1 /= 255
    image_1.shape
    prediction = model.predict(np.array([image_1], np.float32))
    print(prediction)
    return prediction_class[np.argmax(prediction)]

In [15]:
testData = [
    r'./Datasets/test-data/bright_dune/ESP_016576_2570_RED-0061.jpg', 
    r'./Datasets/test-data/dark_dune/ESP_018334_2600_RED-0501.jpg',
    r'./Datasets/test-data/slope_streak/ESP_011289_1950_RED-0067.jpg', 
    r'./Datasets/hirise-map-proj-v3_2/slope_streak/ESP_011289_1950_RED-0230.jpg',
    r'./Datasets/hirise-map-proj-v3_2/crater/ESP_011425_1775_RED-0030-r90.jpg'
]

# findClass(r'./Datasets/test-data/bright_dune/ESP_016576_2570_RED-0061.jpg')

for i in testData:
    print(findClass(i))

[[ 39.074646  37.32895   40.885967 -40.098637 -37.2124  ]]
dark_dune
[[ 10.401164   9.890446  11.020987 -10.728638  -9.871204]]
dark_dune
[[ 46.44769   44.19591   48.579365 -47.500885 -44.00422 ]]
dark_dune
[[ 51.157364  48.644417  53.391365 -52.269226 -48.412342]]
dark_dune
[[ 19.51977   18.394339  20.177359 -19.87043  -18.347044]]
dark_dune
